# Ch 08. 텍스트 분석

## 8.2 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화

In [58]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
from nltk import sent_tokenize
text_sample = ' The Matrix is everywhere its all around us, here even in this room.\
                You can see it out your window or on your television.\
                You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text = text_sample)
print(type(sentences),len(sentences))
print(sentences)

<class 'list'> 3
[' The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


In [17]:
from nltk import word_tokenize

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words),len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [18]:
# 여러 개의 문장으로 된 입력 데이터를 문장 별로 단어 토큰화하게 만드는 함수 생성.
def tokenize_text(text):
    
    # 문장 별로 분리 토큰
    sentences = sent_tokenize(text)
    
    #분리된 문장 별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# 여러 문장에 대해 문장 별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


In [19]:
stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []

# 위 예제에서 3개의 문장 별로 얻은 word_tokens list에 대해 스톱 워드를 제거하는 반복문
for sentence in word_tokens:
    filterd_words = []
    
    # 개별 문장 벼로 토큰화 된 문장 list에 대해 스톱 워드를 제거하는 반복문
    for word in sentence:
        
        #소문자로 모두 변환
        word = word.lower()
        
        #토큰화된 개별 단어가 스톱 워드의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filterd_words.append(word)
    all_tokens.append(filterd_words)
            
print(all_tokens)        

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


In [21]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


In [22]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing', 'v'), lemma.lemmatize('amuses', 'v'),\
      lemma.lemmatize('amused', 'v'))
print(lemma.lemmatize('happier', 'a'), lemma.lemmatize('happiest', 'a'))
print(lemma.lemmatize('fancier', 'a'), lemma.lemmatize('fanciest', 'a'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


amuse amuse amuse
happy happy
fancy fancy


## 8.3 BOW 벡터화를 위한 희소 행렬

In [23]:
import numpy as np

dense = np.array([[3,0,1],[0,2,0]])

In [25]:
from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data,(row_pos, col_pos)))

In [28]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

In [29]:
dense2 = np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]])

# 0이 아닌 데이터 추출
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

# 행 위치와 열 위치를 각각 array로 생성
row_pos = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

# 행 위치 배열의 고유한 값의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13])

# CSR 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [32]:
dense3 = np.array([[0,0,1,0,0,5],
                  [1,4,0,3,2,5],
                  [0,6,0,3,0,0],
                  [2,0,0,0,0,0],
                  [0,0,0,7,0,8],
                  [1,0,0,0,0,0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)

In [33]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [34]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [38]:
import pandas as pd

print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [39]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [41]:
# subset = 'train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),
                  random_state=156)

x_train = train_news.data
y_train = train_news.target

# subset= 'test'로 테스트 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),
                              random_state=156)

x_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),
                                            len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


In [45]:
print(x_train[0])
print(y_train[0])



What I did NOT get with my drive (CD300i) is the System Install CD you
listed as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120
from Direct Express in Chicago (no complaints at all -- good price & good
service).

BTW, I've heard that the System Install CD can be used to boot the mac;
however, my drive will NOT accept a CD caddy is the machine is off.  How can
you boot with it then?

--Dave

4


In [55]:
print(x_test[:5])
print(y_test[:5])

['The tech support line for GCC is 1-800-231-1570.', 'I recently saw a message here (posted by Bob Silverman, I think) which \nreferred to a "birthday" attack on a cryptosystem. I\'m looking for \nreferences on, and explanations of, this type of attack.', 'I cant get through to the author of rtrace. His site is inaccessible\ncan he upload the new version somewhere else please?\n', "Just a quick note on the nwe shape MR2s in the UK.... \n\nWhen they first came out here, there were 3 models. The base model had an \nauto box and engine from the CAMRY 2.0 !!! Well I recentyl found out that this \nmodel is no longer profitable for Toyota and have since scraped it. I've also\nnoticed that auto MR2s have depreciated a lot more than the next model up...", '"Silver Dream Racer" -- Frustrated Brit club racer\'s buddy dies,\nleaving him a built-in-garage "revolutionary, experimental" 500 GP bike.\nBrit club racer uses machine to beat Bad American on Bad Japanese\nFactory Bike at British GP.  Film

## 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

## case 1 : CountVectorizer

In [44]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
x_train_cnt_vect = cnt_vect.fit_transform(x_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피터 벡처화 변환 수행.
x_test_cnt_vect = cnt_vect.transform(x_test)

print('학습 데이터 텍스트의 CountVectorizer Shape :',x_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape : (11314, 101631)


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(x_train_cnt_vect,y_train)
pred = lr_clf.predict(x_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.607


In [61]:
pred[:5]

array([5, 1, 1, 7, 8])

## Case 2 : TfudfVectorizer

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
x_train_tfidf_vect = tfidf_vect.fit_transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

# LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect,y_train)
pred = lr_clf.predict(x_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


## Case 3 : stop words 추가 ngram

In [62]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
x_train_tfidf_vect = tfidf_vect.fit_transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

lr_clf = LogisticRegression()
lr_clf.fit(x_train_tfidf_vect,y_train)
pred = lr_clf.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(
    accuracy_score(y_test,pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


In [63]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv = 3, scoring='accuracy',verbose=1)
grid_cv_lr.fit(x_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가.
pred = grid_cv_lr.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(
    accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

## Case 4 : Case 3 에서 parameter 조정

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
x_train_tfidf_vect = tfidf_vect.fit_transform(x_train)
x_test_tfidf_vect = tfidf_vect.transform(x_test)

lr_clf = LogisticRegression(C = 10)
lr_clf.fit(x_train_tfidf_vect,y_train)
pred = lr_clf.predict(x_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(
    accuracy_score(y_test,pred)))